## Imports / Path

In [1]:
import exifread
import pandas as pd
from exif import Image
from os import walk
path = './'

## Functions 

In [2]:
def get_exif_data(image_file):
    with open(image_file, 'rb') as f:
        exif_tags = exifread.process_file(f, details=False)
        gps_tags = dict()
        for tag in exif_tags.keys():
            if tag in ('GPS GPSLatitude', 'GPS GPSLatitudeRef', 'GPS GPSLongitude', 'GPS GPSLongitudeRef'):
                gps_tags[tag] = exif_tags[tag]
    return gps_tags

def _get_if_exist(data, key):
    if key in data:
        return data[key]

    return None

def _convert_to_degress(value):
    """
    Helper function to convert the GPS coordinates stored in the EXIF to degress in float format
    :param value:
    :type value: exifread.utils.Ratio
    :rtype: float
    """
    d = float(value.values[0].num) / float(value.values[0].den)
    m = float(value.values[1].num) / float(value.values[1].den)
    s = float(value.values[2].num) / float(value.values[2].den)

    return d + (m / 60.0) + (s / 3600.0)

def get_exif_location(exif_data):
    """
    Returns the latitude and longitude, if available, from the provided exif_data (obtained through get_exif_data above)
    """
    lat = None
    lon = None

    gps_latitude = _get_if_exist(exif_data, 'GPS GPSLatitude')
    gps_latitude_ref = _get_if_exist(exif_data, 'GPS GPSLatitudeRef')
    gps_longitude = _get_if_exist(exif_data, 'GPS GPSLongitude')
    gps_longitude_ref = _get_if_exist(exif_data, 'GPS GPSLongitudeRef')

    if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
        lat = _convert_to_degress(gps_latitude)
        if gps_latitude_ref.values[0] != 'N':
            lat = 0 - lat

        lon = _convert_to_degress(gps_longitude)
        if gps_longitude_ref.values[0] != 'E':
            lon = 0 - lon

    return lat, lon

def get_exif_date(file):
    """
    Returns the datetime from exif data 
    """
    with open(file, 'rb') as src:
        img = Image(src)
    date = img.datetime_original
    return date

def wall_data(filenames, path, year):
    """
    Create a dataframe w/ lat, long, date, and filename 
    """
    holder = pd.DataFrame(columns=['Filename', 'Lat', 'Long', 'Date'])
    counter = 0
    for file in filenames:
        if file != '.DS_Store':
            lat, long = get_exif_location(get_exif_data(path+file))
            date = get_exif_date(path+file)
            holder.at[counter,:] = file, lat, long, date
            counter += 1
    holder['Filename'] = f'media/{year}/' + holder['Filename'].astype(str)
    return holder

## Run
Load filenames, process everything vai the wall_data function, then export the csv 


In [3]:
filenames = next(walk(path+'media/2021'), (None, None, []))[2]  # [] if no file
export = wall_data(filenames, './media/2021/', '2021')
export.to_csv('2021.csv')

In [4]:
export

,Filename,Lat,Long,Date
0,media/2021/31CB953F-E1CF-4150-AFE2-9D80A7E8A1E...,44.495578,11.347669,2021:11:19 20:16:09
1,media/2021/IMG_4534.jpeg,44.494092,11.354733,2021:04:13 16:50:45
2,media/2021/IMG_4471.jpeg,44.497833,11.351339,2021:03:23 10:29:18
3,media/2021/IMG_4349.jpeg,44.497139,11.348922,2021:02:13 16:01:01
4,media/2021/IMG_4430.jpeg,44.497983,11.357811,2021:03:16 10:08:48
...,...,...,...,...
195,media/2021/IMG_4389.jpeg,44.494342,11.350364,2021:02:24 10:02:08
196,media/2021/IMG_4223.jpeg,44.495403,11.364436,2021:01:10 11:04:22
197,media/2021/IMG_4370.jpeg,44.498689,11.340464,2021:02:17 18:18:15
198,media/2021/5AE08D95-5261-4D1C-AF19-79F1FE1CFD5...,44.490333,11.352739,2021:08:18 09:45:05
